In [1]:
import neo4j
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_neo4j import init, station_ictag_insert_query
from geofox_client import get_geofox_client
from utils_geofox import get_station_innercityinfo, poisdf2rows
import pandas as pd

c:\Users\paul-\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [5]:
client = get_geofox_client()

In [2]:
# Fetch the POIs
st_ic = get_station_innercityinfo(None, maxList=3000)

In [3]:
st_ic

{'returnCode': 'OK',
 'results': [{'name': 'Hamburg-Harburg',
   'city': 'Hamburg',
   'combinedName': 'Hamburg-Harburg',
   'id': 'Master:8000147',
   'globalId': 'de:02000:8000147',
   'type': 'STATION',
   'coordinate': {'x': 9.990948, 'y': 53.456386},
   'tariffDetails': {'innerCity': False,
    'city': False,
    'cityTraffic': False,
    'gratis': False,
    'greaterArea': True,
    'shVillageId': 56,
    'shTariffZone': 9900,
    'tariffZones': [308],
    'rings': ['B'],
    'fareStage': True,
    'fareStageNumber': -1,
    'tariffNames': ['HVV', 'SH'],
    'uniqueValues': False},
   'serviceTypes': ['bus', 'r', 's', 'train'],
   'hasStationInformation': False},
  {'name': 'Hamburg Hbf',
   'city': 'Hamburg',
   'combinedName': 'Hamburg Hbf',
   'id': 'Master:10950',
   'globalId': 'de:02000:10950',
   'type': 'STATION',
   'coordinate': {'x': 10.007911, 'y': 53.552757},
   'tariffDetails': {'innerCity': True,
    'city': False,
    'cityTraffic': False,
    'gratis': False,
   

In [4]:
st_ic = [s for s in st_ic['results'] if s.get('tariffDetails', {}).get('innerCity', False) is True]

In [5]:
len(st_ic), st_ic[0]

(176,
 {'name': 'Hamburg Hbf',
  'city': 'Hamburg',
  'combinedName': 'Hamburg Hbf',
  'id': 'Master:10950',
  'globalId': 'de:02000:10950',
  'type': 'STATION',
  'coordinate': {'x': 10.007911, 'y': 53.552757},
  'tariffDetails': {'innerCity': True,
   'city': False,
   'cityTraffic': False,
   'gratis': False,
   'greaterArea': True,
   'shVillageId': 1,
   'shTariffZone': 100,
   'tariffZones': [0],
   'rings': ['A'],
   'fareStage': True,
   'fareStageNumber': -1,
   'tariffNames': ['HVV', 'SH'],
   'uniqueValues': False},
  'serviceTypes': ['r', 's', 'train'],
  'hasStationInformation': True})

In [6]:
df_stic = pd.DataFrame(st_ic)

In [7]:
df_stic.columns.to_list()

['name',
 'city',
 'combinedName',
 'id',
 'globalId',
 'type',
 'coordinate',
 'tariffDetails',
 'serviceTypes',
 'hasStationInformation']

In [8]:
impcols = ['id']

In [9]:
df_stic_impcols = df_stic[impcols]

In [10]:
# rename column id to geofoxid
df_stic_impcols.rename(columns={'id': 'geofoxid'}, inplace=True)

C:\Users\paul-\AppData\Local\Temp\ipykernel_23800\2032480438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stic_impcols.rename(columns={'id': 'geofoxid'}, inplace=True)


In [11]:
rows = df_stic_impcols.to_dict(orient='records')

In [12]:
rows

[{'geofoxid': 'Master:10950'},
 {'geofoxid': 'Master:10002'},
 {'geofoxid': 'Master:11007'},
 {'geofoxid': 'Master:11905'},
 {'geofoxid': 'Master:8002548'},
 {'geofoxid': 'Master:11903'},
 {'geofoxid': 'Master:40018'},
 {'geofoxid': 'Master:19232'},
 {'geofoxid': 'Master:43022'},
 {'geofoxid': 'Master:11041'},
 {'geofoxid': 'Master:40950'},
 {'geofoxid': 'Master:49952'},
 {'geofoxid': 'Master:43069'},
 {'geofoxid': 'Master:80900'},
 {'geofoxid': 'Master:80017'},
 {'geofoxid': 'Master:40003'},
 {'geofoxid': 'Master:11005'},
 {'geofoxid': 'Master:55024'},
 {'geofoxid': 'Master:55012'},
 {'geofoxid': 'Master:11046'},
 {'geofoxid': 'Master:43078'},
 {'geofoxid': 'Master:44007'},
 {'geofoxid': 'Master:43042'},
 {'geofoxid': 'Master:54002'},
 {'geofoxid': 'Master:11099'},
 {'geofoxid': 'Master:42011'},
 {'geofoxid': 'Master:55003'},
 {'geofoxid': 'Master:42018'},
 {'geofoxid': 'Master:43068'},
 {'geofoxid': 'Master:11909'},
 {'geofoxid': 'Master:54951'},
 {'geofoxid': 'Master:43019'},
 {'geo

In [15]:
# Execute the query
with driver.session() as session:
    result = session.run(station_ictag_insert_query, rows=rows)
    print(f"{result.single()['total']} stations belong to inner city")

driver.close()

176 stations belong to inner city
